### Import Libraries

In [12]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

### Load data

In [2]:
# Read the dataset
df = pd.read_csv('emotions.csv')
df.head()

,text,label
0,i feel so pissed off over an old friend and so...,anger
1,ive found it has made a huge difference especi...,anger
2,i also feel it is unfortunate that nearly all ...,sadness
3,i feel petty a href http clairee,anger
4,i used to believe that a feeling like fear was...,sadness


In [3]:
df.info

<bound method DataFrame.info of                                                     text    label
0      i feel so pissed off over an old friend and so...    anger
1      ive found it has made a huge difference especi...    anger
2      i also feel it is unfortunate that nearly all ...  sadness
3                       i feel petty a href http clairee    anger
4      i used to believe that a feeling like fear was...  sadness
...                                                  ...      ...
19995  i was i might be buying stuff from there but i...      joy
19996  i like sonam deepika and genelia who i feel ar...      joy
19997  i feel pathetic that i can hardly go a whole d...  sadness
19998  id have spent more time with her on reading i ...  sadness
19999  i do however feel like one of those pathetic g...  sadness

[20000 rows x 2 columns]>

In [4]:
# unique labels
labels_name= df['label'].unique()
labels_name

array(['anger', 'sadness', 'joy', 'love', 'fear', 'surprise'],
      dtype=object)

In [5]:
# Number of classes
num_of_classes = len(labels_name)
print('Total count of the unique labels are ',num_of_classes)

Total count of the unique labels are  6


### Preprocessing

In [6]:
# Tokenization
max_words = 20000
tokenizer = Tokenizer(num_words=max_words,oov_token="<OOV>")

# fit the tokenizer on text data to build vocabulary
tokenizer.fit_on_texts(df['text'])

# Convert text to sequnces of integers
X = tokenizer.texts_to_sequences(df['text'])
print(X[0])

[2, 3, 15, 514, 162, 116, 77, 275, 278, 4, 69, 179]


In [7]:
# Padding the sequences to uniform lengths
maxlen=100
X = pad_sequences(X, padding='post', maxlen=maxlen)
print('Pad sequnce')
print(X)

vocab_size = len(tokenizer.word_index) + 1 # adding 1 for token padding (i.e. index zero)
print('Vocabulary size',vocab_size)


Pad sequnce
[[  2   3  15 ...   0   0   0]
 [ 74 323  13 ...   0   0   0]
 [  2 117   3 ...   0   0   0]
 ...
 [  2   3 580 ...   0   0   0]
 [248  20 405 ...   0   0   0]
 [  2  39 599 ...   0   0   0]]
Vocabulary size 17098


In [8]:
list(tokenizer.word_index.items())[:20]

[('<OOV>', 1),
 ('i', 2),
 ('feel', 3),
 ('and', 4),
 ('to', 5),
 ('the', 6),
 ('a', 7),
 ('feeling', 8),
 ('that', 9),
 ('of', 10),
 ('my', 11),
 ('in', 12),
 ('it', 13),
 ('like', 14),
 ('so', 15),
 ('im', 16),
 ('for', 17),
 ('me', 18),
 ('was', 19),
 ('have', 20)]

In [9]:
# Label encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
print(y)

#One-Hot Encode the labels
y = to_categorical(y)
print(y)

[0 0 4 ... 4 4 4]
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]]


In [10]:
#train,test-split
x_train,x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print(f'Shape of the training data: {x_train.shape},{y_train.shape}')
print(f'Shape of the test data: {x_test.shape},{y_test.shape}')
print(f'Shape of the test data: {x_val.shape},{y_val.shape}')


Shape of the training data: (12800, 100),(12800, 6)
Shape of the test data: (4000, 100),(4000, 6)
Shape of the test data: (3200, 100),(3200, 6)


### Create model

In [13]:
embedding_dim = 100
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen),
    LSTM(128, activation='tanh', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Flatten(),  
    Dense(num_of_classes, activation='softmax')
])


model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


### Train the model

In [14]:
history = model.fit(x_train, y_train, epochs=10,batch_size=32, validation_data=(x_val, y_val))


Epoch 1/10


400/400 [==============================] - 69s 160ms/step - loss: 1.7271 - accuracy: 0.3187 - val_loss: 1.5913 - val_accuracy: 0.3344
Epoch 2/10
400/400 [==============================] - 64s 160ms/step - loss: 1.5825 - accuracy: 0.3297 - val_loss: 1.5913 - val_accuracy: 0.3344
Epoch 3/10
400/400 [==============================] - 71s 177ms/step - loss: 1.5788 - accuracy: 0.3320 - val_loss: 1.5942 - val_accuracy: 0.3344
Epoch 4/10
400/400 [==============================] - 72s 181ms/step - loss: 1.5778 - accuracy: 0.3360 - val_loss: 1.5903 - val_accuracy: 0.3344
Epoch 5/10
168/400 [===========>..................] - ETA: 34s - loss: 1.5741 - accuracy: 0.3341

KeyboardInterrupt: 

In [ ]:
plt.hist(df['text'].apply(lambda x: len(x.split())))
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.show()


In [ ]:
df['label'].value_counts().plot(kind='bar')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.show()


### Visualize the model performance

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


### Evaluate on the test set

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

In [ ]:
# Make predictions on the test set
predictions = model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

# Print classification report
print('\nClassification Report:')
print(classification_report(true_labels, predicted_labels))

In [ ]:
# Plot confusion matrix
cf_matrix = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(8, 8))
plt.imshow(cf_matrix, cmap='Blues', interpolation='nearest')
plt.title('Confusion Matrix')
plt.xticks(np.arange(num_of_classes), labels_name)  
plt.yticks(np.arange(num_of_classes), labels_name)  
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()